<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
Introduction to Pandas
</p><br>

**pandas** is a Python package providing fast, flexible, and expressive data structures designed to work with *relational* or *labeled* data both. It is a fundamental high-level building block for doing practical, real world data analysis in Python. 

pandas is well suited for:

- Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet
- Ordered and unordered (not necessarily fixed-frequency) time series data.
- Arbitrary matrix data (homogeneously typed or heterogeneous) with row and column labels
- Any other form of observational / statistical data sets. The data actually need not be labeled at all to be placed into a pandas data structure


Key features:
    
- Easy handling of **missing data**
- **Size mutability**: columns can be inserted and deleted from DataFrame and higher dimensional objects
- Automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the data can be aligned automatically
- Powerful, flexible **group by functionality** to perform split-apply-combine operations on data sets
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- Intuitive **merging and joining** data sets
- Flexible **reshaping and pivoting** of data sets
- **Hierarchical labeling** of axes
- Robust **IO tools** for loading data from flat files, Excel files, databases, and HDF5
- **Time series functionality**: date range generation and frequency conversion, moving window statistics, moving window linear regressions, date shifting and lagging, etc.


# The Series Data Structure

![Understanding%20Data%20Series%20-%2001.PNG](DataSeries.png)

## Creating Series 

In [1]:
import pandas as pd
#pd.Series?

We can create a series by passing in a list of values. 
When we do this, Pandas automatically assigns an index starting with zero and sets the name of the series to None.

In [2]:
animals = ['Tiger', 'Elephant','Deer','Leopard']
pd.Series(animals)

0       Tiger
1    Elephant
2        Deer
3     Leopard
dtype: object

In [3]:
numbers = [1,2,3,4,5]
pd.Series(numbers)

0    1
1    2
2    3
3    4
4    5
dtype: int64

Series can be created from dictionary data. 
If we do this, the index is automatically assigned to the keys of the dictionary that we provided and not just incrementing integers.

In [4]:
sports = {'Archery':'Bhutan','Golf':'Scotland', 'Sumo':'Japan', 'Taekwando':'Korea'}
s = pd.Series(sports)
print(s)

Archery        Bhutan
Golf         Scotland
Sumo            Japan
Taekwando       Korea
dtype: object


To get the index objects

In [5]:
s.index

Index(['Archery', 'Golf', 'Sumo', 'Taekwando'], dtype='object')

We can also separate our index creation from the data by passing in the index as a list explicitly to the series 

In [6]:
a = pd.Series(['Tiger', 'Elephant','Bear','Moose'],index=['India','India','America','Canada'])
a

India         Tiger
India      Elephant
America        Bear
Canada        Moose
dtype: object

**What happens if our list of values in the index object are not aligned with the keys in our dictionary for creating the series?**

Pandas overrides all of the indices values that we provided.

 1. It will ignore from our dictionary, all keys, which are not in our index, and 
 2. pandas will add none type or NAN values for any index value you provide, which is not in your dictionary key list.

In [7]:
sports = {'Archery': 'Bhutan',
          'Golf': 'Scotland',
          'Sumo': 'Japan',
          'Taekwondo': 'South Korea'}
# ignores the key, Archery and Taekwando as it is not in the index provided below
# added non type or NaN values for the new index 'Hockey'
s = pd.Series(sports, index=['Golf', 'Sumo', 'Hockey'])
s

Golf      Scotland
Sumo         Japan
Hockey         NaN
dtype: object

## Querying a Series 

A panda Series can be queried, either by the **index position** or the **index label**.

In [3]:
sports = {'Archery': 'Bhutan',
          'Golf': 'Scotland',
          'Sumo': 'Japan',
          'Taekwondo': 'South Korea'}
s = pd.Series(sports)
s

Archery           Bhutan
Golf            Scotland
Sumo               Japan
Taekwondo    South Korea
dtype: object

To query by numeric location, starting at zero, use the **iloc** attribute. 

In [9]:
s.iloc[3]

'South Korea'

To query by the index label, you can use the **loc** attribute

In [10]:
s.loc['Golf']

'Scotland'

If we pass in an integer parameter, the operator will behave as if we want it to query via the iloc attribute. 
If we pass in an object, it will query as if we wanted to use the label based loc attribute. 

In [11]:
s[3] 

'South Korea'

In [12]:
s['Golf']

'Scotland'

Some filtering options


In [4]:
bacteria = pd.Series([632, 1638, 569, 115], 
    index=['Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes'])

bacteria

Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
dtype: int64

In [14]:
bacteria[[name.endswith('bacteria') for name in bacteria.index]]

Proteobacteria    1638
Actinobacteria     569
dtype: int64

In [5]:
bacteria>1000

Firmicutes        False
Proteobacteria     True
Actinobacteria    False
Bacteroidetes     False
dtype: bool

In [15]:
bacteria[bacteria>1000]

Proteobacteria    1638
dtype: int64

**Vectorization** - Pandas and the underlying NumPy libraries support a method of computation called vectorization. The practice of replacing explicit loops  with array expressions is commonly referred to as vectorization.

Vectorization works with most of the functions in the NumPy library, including the sum function. 

In [16]:
prices = pd.Series([100.00, 120.00, 101.00, 3.00])
prices

0    100.0
1    120.0
2    101.0
3      3.0
dtype: float64

In [17]:
# the below approach of iterating over the items of the series and invoking the operation of addition works but is slow
total = 0
for item in prices:
    total+=item
print(total)

324.0


In [18]:
import numpy as np
# the sum function can take any iterable item, in this case a Series
total = np.sum(prices)
print(total)

324.0


In [19]:
#this creates a big series of random numbers
s = pd.Series(np.random.randint(0,1000,10000))
s.head()

0    618
1    798
2    835
3    384
4    529
dtype: int32

In [20]:
len(s)

10000

In [21]:
%%timeit -n 100
summary = 0
for item in s:
    summary+=item

100 loops, best of 3: 982 µs per loop


In [22]:
%%timeit -n 100
summary = np.sum(s)

100 loops, best of 3: 134 µs per loop


In [23]:
%%timeit -n 10
s = pd.Series(np.random.randint(0,1000,10000))
for label, value in s.iteritems():
    s.loc[label]= value+2

10 loops, best of 3: 700 ms per loop


In [24]:
%%timeit -n 10
s = pd.Series(np.random.randint(0,1000,10000))
s+=2

10 loops, best of 3: 363 µs per loop


# The DataFrame

Inevitably, we want to be able to store, view and manipulate data that is *multivariate*, where for every index there are multiple fields or columns of data, often of varying data type.

A `DataFrame` is a tabular data structure, encapsulating multiple series like columns in a spreadsheet. Data are stored internally as a 2-dimensional object, but the `DataFrame` allows us to represent and manipulate higher-dimensional data.

![Understanding%20DataFrame%20-%2001.PNG](DataFrame.png)

In [6]:
import pandas as pd

In [7]:
## We create a group of Series, where each Series represents a row of Data 
purchase_1 = pd.Series({'FirstName': 'Nimish','LastName': 'Sanghi','Item Purchased':'Dog Food','Cost':'100.50'})
purchase_2 = pd.Series({'FirstName': 'Mukund','LastName': 'Goel','Item Purchased':'Cat Food','Cost':'80.50'})
purchase_3 = pd.Series({'FirstName': 'Piush','LastName': 'Sharma','Item Purchased':'Bird Food','Cost':'50.50'})

In [8]:
## Feed the Series as the first argument of the DataFrame 
## index values as which Store the purchase is done
## the indices and column names along either axes, horizontal or vertical, could be non-unique

df = pd.DataFrame([purchase_1,purchase_2,purchase_3],index =['Store 1', 'Store 2','Store 1'])

In [9]:
df

,Cost,FirstName,Item Purchased,LastName
Store 1,100.50,Nimish,Dog Food,Sanghi
Store 2,80.50,Mukund,Cat Food,Goel
Store 1,50.50,Piush,Bird Food,Sharma


In [10]:
## Similar to the series, we can extract data using the iLoc and Loc attributes. 
## loc and iloc are used for row selection. 
df.loc['Store 2']

Cost                 80.50
FirstName           Mukund
Item Purchased    Cat Food
LastName              Goel
Name: Store 2, dtype: object

In [32]:
df.loc['Store 1']

,Cost,FirstName,Item Purchased,LastName
Store 1,100.50,Nimish,Dog Food,Sanghi
Store 1,50.50,Piush,Bird Food,Sharma


In [35]:
## to Find the Store 1 cost 
## loc can take 2 parameters row index and list of column names
df.loc['Store 1', ['Cost']]

,Cost
Store 1,100.50
Store 1,50.50


In [36]:
## to Find the Store 1 cost and item purchased
df.loc['Store 1', ['Item Purchased','Cost']]

,Item Purchased,Cost
Store 1,Dog Food,100.50
Store 1,Bird Food,50.50


In [37]:
## loc support slicing
df.loc[:,['Item Purchased','Cost']]

,Item Purchased,Cost
Store 1,Dog Food,100.50
Store 2,Cat Food,80.50
Store 1,Bird Food,50.50


In [38]:
# If the column name is a string we can get the data for a column in the following 2 ways
df['FirstName']

Store 1    Nimish
Store 2    Mukund
Store 1     Piush
Name: FirstName, dtype: object

In [39]:
df.FirstName

Store 1    Nimish
Store 2    Mukund
Store 1     Piush
Name: FirstName, dtype: object

## Adding a column

We can add a column to the dataframe as below 

In [40]:
df['Catagory'] = 'Premium'

In [41]:
df

,Cost,FirstName,Item Purchased,LastName,Catagory
Store 1,100.50,Nimish,Dog Food,Sanghi,Premium
Store 2,80.50,Mukund,Cat Food,Goel,Premium
Store 1,50.50,Piush,Bird Food,Sharma,Premium


If we want to assign a new column value, where each cell is derived from the values already in its row we can do it as below

We have to pass axis=1 so that the function gets applied across each row instead of each column

In [42]:
df['Name'] = df.apply(lambda row: row['FirstName'] + ' ' + row['LastName'],axis=1)

In [43]:
df

,Cost,FirstName,Item Purchased,LastName,Catagory,Name
Store 1,100.50,Nimish,Dog Food,Sanghi,Premium,Nimish Sanghi
Store 2,80.50,Mukund,Cat Food,Goel,Premium,Mukund Goel
Store 1,50.50,Piush,Bird Food,Sharma,Premium,Piush Sharma


## Dropping data in DataFrame 

In [44]:
## drop takes single index/row label 
## The drop function doesn't change the DataFrame by default. 
## And instead, returns to you a copy of the DataFrame with the given rows removed.
df.drop('Store 2')

,Cost,FirstName,Item Purchased,LastName,Catagory,Name
Store 1,100.50,Nimish,Dog Food,Sanghi,Premium,Nimish Sanghi
Store 1,50.50,Piush,Bird Food,Sharma,Premium,Piush Sharma


In [45]:
## The original DataFrame is still intact
df

,Cost,FirstName,Item Purchased,LastName,Catagory,Name
Store 1,100.50,Nimish,Dog Food,Sanghi,Premium,Nimish Sanghi
Store 2,80.50,Mukund,Cat Food,Goel,Premium,Mukund Goel
Store 1,50.50,Piush,Bird Food,Sharma,Premium,Piush Sharma


## Indexing Dataframes

In [46]:
df = pd.read_csv('census.csv')
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


The dataset  contains data at both State and County level

In [47]:
df['SUMLEV'].unique()

array([40, 50], dtype=int64)

We are interested only in the County level information

In [48]:
df=df[df['SUMLEV'] == 50]
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


We are only interested in few of the columns

In [49]:
columns_to_keep = ['STNAME',
                   'CTYNAME',
                   'BIRTHS2010',
                   'BIRTHS2011',
                   'BIRTHS2012',
                   'BIRTHS2013',
                   'BIRTHS2014',
                   'BIRTHS2015',
                   'POPESTIMATE2010',
                   'POPESTIMATE2011',
                   'POPESTIMATE2012',
                   'POPESTIMATE2013',
                   'POPESTIMATE2014',
                   'POPESTIMATE2015']
df = df[columns_to_keep]
df.head()

,STNAME,CTYNAME,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
1,Alabama,Autauga County,151,636,615,574,623,600,54660,55253,55175,55038,55290,55347
2,Alabama,Baldwin County,517,2187,2092,2160,2186,2240,183193,186659,190396,195126,199713,203709
3,Alabama,Barbour County,70,335,300,283,260,269,27341,27226,27159,26973,26815,26489
4,Alabama,Bibb County,44,266,245,259,247,253,22861,22733,22642,22512,22549,22583
5,Alabama,Blount County,183,744,710,646,618,603,57373,57711,57776,57734,57658,57673


In [50]:
df = df.set_index(['STNAME', 'CTYNAME'])
df.head()

BIRTHS2010  BIRTHS2011  BIRTHS2012  BIRTHS2013  \
STNAME  CTYNAME                                                          
Alabama Autauga County         151         636         615         574   
        Baldwin County         517        2187        2092        2160   
        Barbour County          70         335         300         283   
        Bibb County             44         266         245         259   
        Blount County          183         744         710         646   

                        BIRTHS2014  BIRTHS2015  POPESTIMATE2010  \
STNAME  CTYNAME                                                   
Alabama Autauga County         623         600            54660   
        Baldwin County        2186        2240           183193   
        Barbour County         260         269            27341   
        Bibb County            247         253            22861   
        Blount County          618         603            57373   

                        POPESTIMATE2011  POPESTIMATE2012  POPESTIMATE2013  \
STNAME  CTYNAME                                                             
Alabama Autauga County            55253            55175            55038   
        Baldwin County           186659           190396           195126   
        Barbour County            27226            27159            26973   
        Bibb County               22733            22642            22512   
        Blount County             57711            57776            57734   

                        POPESTIMATE2014  POPESTIMATE2015  
STNAME  CTYNAME                                           
Alabama Autauga County            55290            55347  
        Baldwin County           199713           203709  
        Barbour County            26815            26489  
        Bibb County               22549            22583  
        Blount County             57658            57673

Ways to query the grouped data

 loc attribute of the DataFrame can take multiple arguments. And it could query both the row and the columns. When you use a MultiIndex, you must provide the arguments in order by the level you wish to query. Inside of the index, each column is called a level and the outermost column is level zero. For instance, if we want to see the population results from Washtenaw County, you'd want the first argument as the state of Michigan. 

In [51]:
df.loc['Michigan', 'Washtenaw County']

BIRTHS2010            977
BIRTHS2011           3826
BIRTHS2012           3780
BIRTHS2013           3662
BIRTHS2014           3683
BIRTHS2015           3709
POPESTIMATE2010    345563
POPESTIMATE2011    349048
POPESTIMATE2012    351213
POPESTIMATE2013    354289
POPESTIMATE2014    357029
POPESTIMATE2015    358880
Name: (Michigan, Washtenaw County), dtype: int64

You might be interested in just comparing two counties. For instance, Washtenaw and Wayne County. To do this, we can pass the loc method, a list of tuples which describe the indices we wish to query. Since we have a MultiIndex of two values, the state and the county, we need to provide two values as each element of our filtering list. 

In [52]:
df.loc[ [('Michigan', 'Washtenaw County'),
         ('Michigan', 'Wayne County')] ]

BIRTHS2010  BIRTHS2011  BIRTHS2012  BIRTHS2013  \
STNAME   CTYNAME                                                            
Michigan Washtenaw County         977        3826        3780        3662   
         Wayne County            5918       23819       23270       23377   

                           BIRTHS2014  BIRTHS2015  POPESTIMATE2010  \
STNAME   CTYNAME                                                     
Michigan Washtenaw County        3683        3709           345563   
         Wayne County           23607       23586          1815199   

                           POPESTIMATE2011  POPESTIMATE2012  POPESTIMATE2013  \
STNAME   CTYNAME                                                               
Michigan Washtenaw County           349048           351213           354289   
         Wayne County              1801273          1792514          1775713   

                           POPESTIMATE2014  POPESTIMATE2015  
STNAME   CTYNAME                                             
Michigan Washtenaw County           357029           358880  
         Wayne County              1766008          1759335

# Missing Values

In [53]:
df = pd.read_csv('log.csv')
df

,time,user,video,playback position,paused,volume
0,1469974424,cheryl,intro.html,5,False,10.0
1,1469974454,cheryl,intro.html,6,NaN,NaN
2,1469974544,cheryl,intro.html,9,NaN,NaN
3,1469974574,cheryl,intro.html,10,NaN,NaN
4,1469977514,bob,intro.html,1,NaN,NaN
5,1469977544,bob,intro.html,1,NaN,NaN
6,1469977574,bob,intro.html,1,NaN,NaN
7,1469977604,bob,intro.html,1,NaN,NaN
8,1469974604,cheryl,intro.html,11,NaN,NaN
9,1469974694,cheryl,intro.html,14,NaN,NaN


In Pandas we can sort either by index or by values. Here we'll just promote the time stamp to an index then sort on the index. 

In [54]:
df = df.set_index('time')
df = df.sort_index()
df

,user,video,playback position,paused,volume
time,,,,,
1469974424,cheryl,intro.html,5,False,10.0
1469974424,sue,advanced.html,23,False,10.0
1469974454,cheryl,intro.html,6,NaN,NaN
1469974454,sue,advanced.html,24,NaN,NaN
1469974484,cheryl,intro.html,7,NaN,NaN
1469974514,cheryl,intro.html,8,NaN,NaN
1469974524,sue,advanced.html,25,NaN,NaN
1469974544,cheryl,intro.html,9,NaN,NaN
1469974554,sue,advanced.html,26,NaN,NaN


If we look closely at the output though we'll notice that the index isn't really unique. Two users seem to be able to use the system at the same time. Again, a very common case. 

Let's reset the index, and use some multi-level indexing instead, and promote the user name to a second level of the index to deal with that issue

In [55]:
df = df.reset_index()
df = df.set_index(['time', 'user'])
df

video  playback position paused  volume
time       user                                                   
1469974424 cheryl     intro.html                  5  False    10.0
           sue     advanced.html                 23  False    10.0
1469974454 cheryl     intro.html                  6    NaN     NaN
           sue     advanced.html                 24    NaN     NaN
1469974484 cheryl     intro.html                  7    NaN     NaN
1469974514 cheryl     intro.html                  8    NaN     NaN
1469974524 sue     advanced.html                 25    NaN     NaN
1469974544 cheryl     intro.html                  9    NaN     NaN
1469974554 sue     advanced.html                 26    NaN     NaN
1469974574 cheryl     intro.html                 10    NaN     NaN
1469974604 cheryl     intro.html                 11    NaN     NaN
1469974624 sue     advanced.html                 27    NaN     NaN
1469974634 cheryl     intro.html                 12    NaN     NaN
1469974654 sue     advanced.html                 28    NaN     5.0
1469974664 cheryl     intro.html                 13    NaN     NaN
1469974694 cheryl     intro.html                 14    NaN     NaN
1469974724 cheryl     intro.html                 15    NaN     NaN
           sue     advanced.html                 29    NaN     NaN
1469974754 sue     advanced.html                 30    NaN     NaN
1469974824 sue     advanced.html                 31    NaN     NaN
1469974854 sue     advanced.html                 32    NaN     NaN
1469974924 sue     advanced.html                 33    NaN     NaN
1469977424 bob        intro.html                  1   True    10.0
1469977454 bob        intro.html                  1    NaN     NaN
1469977484 bob        intro.html                  1    NaN     NaN
1469977514 bob        intro.html                  1    NaN     NaN
1469977544 bob        intro.html                  1    NaN     NaN
1469977574 bob        intro.html                  1    NaN     NaN
1469977604 bob        intro.html                  1    NaN     NaN
1469977634 bob        intro.html                  1    NaN     NaN
1469977664 bob        intro.html                  1    NaN     NaN
1469977694 bob        intro.html                  1    NaN     NaN
1469977724 bob        intro.html                  1    NaN     NaN

One of the handy functions that Pandas has for working with missing values is the filling function, fillna. This function takes a number or parameters, for instance, you could pass in a single value which is called a scalar value to change all of the missing data to one value. This isn't really applicable in this case, but it's a pretty common use case. 

Next up though is the method parameter. The two common fill values are ffill and bfill. ffill is for forward filling and it updates a NaN value for a particular cell with the value from the previous row. It's important to note that your data needs to be sorted in order for this to have the effect you might want. Data that comes from traditional database management systems usually has no order guarantee, just like this data.

In [56]:
df = df.fillna(method='ffill')
df.head()

video  playback position paused  volume
time       user                                                   
1469974424 cheryl     intro.html                  5  False    10.0
           sue     advanced.html                 23  False    10.0
1469974454 cheryl     intro.html                  6  False    10.0
           sue     advanced.html                 24  False    10.0
1469974484 cheryl     intro.html                  7  False    10.0

## Merging Dataframes

![Understanding%20Merge%20-%2001.PNG](Understanding%20Merge%20-%2001.PNG)

In [58]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                         {'Name': 'Sally', 'Role': 'Course liasion'},
                         {'Name': 'James', 'Role': 'Grader'}])
staff_df = staff_df.set_index('Name')
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business'},
                           {'Name': 'Mike', 'School': 'Law'},
                           {'Name': 'Sally', 'School': 'Engineering'}])
student_df = student_df.set_index('Name')
print(staff_df.head())
print()
print(student_df.head())

                 Role
Name                 
Kelly  Director of HR
Sally  Course liasion
James          Grader

            School
Name              
James     Business
Mike           Law
Sally  Engineering


In [59]:
# UNION in Set Theory or Full Outer Join in Database Terminology  
pd.merge(staff_df, student_df, how='outer', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Kelly,Director of HR,NaN
Mike,NaN,Law
Sally,Course liasion,Engineering


In [60]:
# INTERSECTION in Set Theory or Inner Join in Database Terminology  
pd.merge(staff_df, student_df, how='inner', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Sally,Course liasion,Engineering


In [61]:
# We don't need to use indices to join on, we can use columns as well. Here's an example. 
# Multi indexing and multiple columns
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 'Role': 'Grader'}])
student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 'School': 'Engineering'}])
print(staff_df)
print(' ')
print(student_df)
pd.merge(staff_df, student_df, how='inner', left_on=['First Name','Last Name'], right_on=['First Name','Last Name'])

  First Name   Last Name            Role
0      Kelly  Desjardins  Director of HR
1      Sally      Brooks  Course liasion
2      James       Wilde          Grader
 
  First Name Last Name       School
0      James   Hammond     Business
1       Mike     Smith          Law
2      Sally    Brooks  Engineering


,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering


## Method Chaining

In [62]:
import pandas as pd
df = pd.read_csv('census.csv')
#df.head()

In [63]:
(df.where(df['SUMLEV']==50)
    .dropna()
    .set_index(['STNAME','CTYNAME'])
    .rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'}))

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME    CTYNAME                                                       
Alabama   Autauga County        50.0     3.0       6.0    1.0     1.0   
          Baldwin County        50.0     3.0       6.0    1.0     3.0   
          Barbour County        50.0     3.0       6.0    1.0     5.0   
          Bibb County           50.0     3.0       6.0    1.0     7.0   
          Blount County         50.0     3.0       6.0    1.0     9.0   
          Bullock County        50.0     3.0       6.0    1.0    11.0   
          Butler County         50.0     3.0       6.0    1.0    13.0   
          Calhoun County        50.0     3.0       6.0    1.0    15.0   
          Chambers County       50.0     3.0       6.0    1.0    17.0   
          Cherokee County       50.0     3.0       6.0    1.0    19.0   
          Chilton County        50.0     3.0       6.0    1.0    21.0   
          Choctaw County        50.0     3.0       6.0    1.0    23.0   
          Clarke County         50.0     3.0       6.0    1.0    25.0   
          Clay County           50.0     3.0       6.0    1.0    27.0   
          Cleburne County       50.0     3.0       6.0    1.0    29.0   
          Coffee County         50.0     3.0       6.0    1.0    31.0   
          Colbert County        50.0     3.0       6.0    1.0    33.0   
          Conecuh County        50.0     3.0       6.0    1.0    35.0   
          Coosa County          50.0     3.0       6.0    1.0    37.0   
          Covington County      50.0     3.0       6.0    1.0    39.0   
          Crenshaw County       50.0     3.0       6.0    1.0    41.0   
          Cullman County        50.0     3.0       6.0    1.0    43.0   
          Dale County           50.0     3.0       6.0    1.0    45.0   
          Dallas County         50.0     3.0       6.0    1.0    47.0   
          DeKalb County         50.0     3.0       6.0    1.0    49.0   
          Elmore County         50.0     3.0       6.0    1.0    51.0   
          Escambia County       50.0     3.0       6.0    1.0    53.0   
          Etowah County         50.0     3.0       6.0    1.0    55.0   
          Fayette County        50.0     3.0       6.0    1.0    57.0   
          Franklin County       50.0     3.0       6.0    1.0    59.0   
...                              ...     ...       ...    ...     ...   
Wisconsin Washburn County       50.0     2.0       3.0   55.0   129.0   
          Washington County     50.0     2.0       3.0   55.0   131.0   
          Waukesha County       50.0     2.0       3.0   55.0   133.0   
          Waupaca County        50.0     2.0       3.0   55.0   135.0   
          Waushara County       50.0     2.0       3.0   55.0   137.0   
          Winnebago County      50.0     2.0       3.0   55.0   139.0   
          Wood County           50.0     2.0       3.0   55.0   141.0   
Wyoming   Albany County         50.0     4.0       8.0   56.0     1.0   
          Big Horn County       50.0     4.0       8.0   56.0     3.0   
          Campbell County       50.0     4.0       8.0   56.0     5.0   
          Carbon County         50.0     4.0       8.0   56.0     7.0   
          Converse County       50.0     4.0       8.0   56.0     9.0   
          Crook County          50.0     4.0       8.0   56.0    11.0   
          Fremont County        50.0     4.0       8.0   56.0    13.0   
          Goshen County         50.0     4.0       8.0   56.0    15.0   
          Hot Springs County    50.0     4.0       8.0   56.0    17.0   
          Johnson County        50.0     4.0       8.0   56.0    19.0   
          Laramie County        50.0     4.0       8.0   56.0    21.0   
          Lincoln County        50.0     4.0       8.0   56.0    23.0   
          Natrona County        50.0     4.0       8.0   56.0    25.0   
          Niobrara County       50.0     4.0       8.0   56.0    27.0   
          Park County           50.0     4.0       8.0   56.0    29.0   
          Platt

In [68]:
for group, frame in df.groupby('STNAME'):
    print(frame['STNAME','CTYNAME','CENSUS2010POP'].head())

KeyError: ('STNAME', 'CTYNAME', 'CENSUS2010POP')

## Group by

In [69]:
for group, frame in df.groupby('STNAME'):
    avg = np.average(frame['CENSUS2010POP'])
    print('Counties in state ' + group + ' have an average population of ' + str(avg))

Counties in state Alabama have an average population of 140580.470588
Counties in state Alaska have an average population of 47348.7333333
Counties in state Arizona have an average population of 799002.125
Counties in state Arkansas have an average population of 76734.6842105
Counties in state California have an average population of 1262845.9661
Counties in state Colorado have an average population of 154744.492308
Counties in state Connecticut have an average population of 794243.777778
Counties in state Delaware have an average population of 448967.0
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 552979.705882
Counties in state Georgia have an average population of 121095.6625
Counties in state Hawaii have an average population of 453433.666667
Counties in state Idaho have an average population of 69670.3111111
Counties in state Illinois have an average population of 249138.485437
Counties in stat

**groupby** object also has a method called agg which is short for aggregate. This method applies a function to the column or columns of data in the group, and returns the results. 

With agg, you simply pass in a dictionary of the column names that you're interested in, and the function that you want to apply. For instance to build a summary data frame for the average populations per state, we could just give agg a dictionary with the Census 2010 pop key and the numpy average function. 

In [70]:
df = pd.read_csv('census.csv')
df = df[df['SUMLEV']==50]

In [71]:
df.groupby('STNAME').agg({'CENSUS2010POP': np.average})

,CENSUS2010POP
STNAME,
Alabama,71339.343284
Alaska,24490.724138
Arizona,426134.466667
Arkansas,38878.906667
California,642309.586207
Colorado,78581.187500
Connecticut,446762.125000
Delaware,299311.333333
District of Columbia,601723.000000


## Importing Data 
Most of the times we will use Pandas to load data and immediately convert it into Numpy array

In [72]:
url='https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
df = pd.read_csv(url,header = None) # By defaultpandas reads the header
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [73]:
# Preview of the data set
df.head(10)
#df.tail()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
5,2,?,audi,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250
6,1,158,audi,gas,std,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,17710
7,1,?,audi,gas,std,four,wagon,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,18920
8,1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.40,8.3,140,5500,17,20,23875
9,0,?,audi,gas,turbo,two,hatchback,4wd,front,99.5,...,131,mpfi,3.13,3.40,7.0,160,5500,16,22,?


In [74]:
# Adding Headers for the column names
headers = ['symboling','normalized-losses','make','fuel-type','aspiration','num-of-doors','body-style','drive-wheels','engine-location','wheel-base','length','width', 'height','curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore','stroke','compresssion-ratio','horse-power','peak-rpm','city-mpg','highway-mpg','price']
df.columns=headers
df.head(5)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compresssion-ratio,horse-power,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [75]:
df.dtypes

symboling               int64
normalized-losses      object
make                   object
fuel-type              object
aspiration             object
num-of-doors           object
body-style             object
drive-wheels           object
engine-location        object
wheel-base            float64
length                float64
width                 float64
height                float64
curb-weight             int64
engine-type            object
num-of-cylinders       object
engine-size             int64
fuel-system            object
bore                   object
stroke                 object
compresssion-ratio    float64
horse-power            object
peak-rpm               object
city-mpg                int64
highway-mpg             int64
price                  object
dtype: object

In [76]:
# To get the basic statistical summary
df.describe()

,symboling,wheel-base,length,width,height,curb-weight,engine-size,compresssion-ratio,city-mpg,highway-mpg
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,10.142537,25.219512,30.751220
std,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,3.972040,6.542142,6.886443
min,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,7.000000,13.000000,16.000000
25%,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,8.600000,19.000000,25.000000
50%,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,9.000000,24.000000,30.000000
75%,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,9.400000,30.000000,34.000000
max,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,23.000000,49.000000,54.000000


In [77]:
df.describe(include='all')

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compresssion-ratio,horse-power,peak-rpm,city-mpg,highway-mpg,price
count,205.000000,205,205,205,205,205,205,205,205,205.000000,...,205.000000,205,205,205,205.000000,205,205,205.000000,205.000000,205
unique,NaN,52,22,2,2,3,5,3,2,NaN,...,NaN,8,39,37,NaN,60,24,NaN,NaN,187
top,NaN,?,toyota,gas,std,four,sedan,fwd,front,NaN,...,NaN,mpfi,3.62,3.40,NaN,68,5500,NaN,NaN,?
freq,NaN,41,32,185,168,114,96,120,202,NaN,...,NaN,94,23,20,NaN,19,37,NaN,NaN,4
mean,0.834146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.756585,...,126.907317,NaN,NaN,NaN,10.142537,NaN,NaN,25.219512,30.751220,NaN
std,1.245307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.021776,...,41.642693,NaN,NaN,NaN,3.972040,NaN,NaN,6.542142,6.886443,NaN
min,-2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.600000,...,61.000000,NaN,NaN,NaN,7.000000,NaN,NaN,13.000000,16.000000,NaN
25%,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.500000,...,97.000000,NaN,NaN,NaN,8.600000,NaN,NaN,19.000000,25.000000,NaN
50%,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.000000,...,120.000000,NaN,NaN,NaN,9.000000,NaN,NaN,24.000000,30.000000,NaN
75%,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.400000,...,141.000000,NaN,NaN,NaN,9.400000,NaN,NaN,30.000000,34.000000,NaN


## Exporting a Pandas dataframe to CSV

In [78]:
path = 'C:\\Users\\soais\\automobile.csv'
df.to_csv(path)